In [1]:

import numpy as np 
import pandas as pd 



import os


In [2]:
#importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string, os
import nltk
import re
import keras
import random
import io
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adamax
import sys
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image, ImageDraw, ImageFont
import warnings
warnings.filterwarnings("ignore")


In [3]:
data = pd.read_csv('my_data.csv')
data.head()

,Unnamed: 0.1,Nom,Artiste,Genre,Annee,Paroles,country_mb
0,0,As It Was,Harry Styles,pop,2022,[Intro] Come on Harry we wanna say goodn...,United Kingdom
1,1,Heat Waves,Glass Animals,pop,2022,[Intro] (Last night all I think about is y...,United Kingdom
2,2,STAY (with Justin Bieber),The Kid LAROI,hip hop,2022,[Chorus: The Kid LAROI] I do the same thing ...,NaN
3,3,Me Porto Bonito,Bad Bunny,reggae,2022,"[Letra de ""Me Porto Bonito""] [Intro: Bad ...",United States
4,4,Tití Me Preguntó,Bad Bunny,reggae,2022,"[Letra de ""Tití Me Preguntó""] [Intro: Bad...",United States


In [4]:
#Printing the names of artists in the lyrics data
print("Artists in the data:\n",data.Artiste.value_counts()) 

Artists in the data:
 Eminem             27
Taylor Swift       25
P!nk               24
Various Artists    24
Ed Sheeran         22
                   ..
Emeli Sandé         1
Grimes              1
DEV                 1
Myke Towers         1
Donell Jones        1
Name: Artiste, Length: 899, dtype: int64


In [5]:
#Printing the size of dataset
print("Size of Dataset:",data.shape)

Size of Dataset: (2450, 7)


In [6]:
data.drop(['Unnamed: 0.1', 'Genre', 'Annee','country_mb'],axis=1,inplace=True)

In [7]:
#Adding a column of numbers of Characters,words and sentences in each msg
data["No_of_Characters"] = data["Paroles"].astype(str).apply(len)
data["No_of_Words"] = data["Paroles"].astype(str).apply(lambda x: len(nltk.word_tokenize(x)))
data["No_of_Lines"] = data["Paroles"].astype(str).str.split('\n').apply(len)
data.describe()

,No_of_Characters,No_of_Words,No_of_Lines
count,2450.000000,2450.000000,2450.0
mean,1469.992245,334.172245,1.0
std,1834.613931,451.645010,0.0
min,1.000000,1.000000,1.0
25%,919.250000,192.000000,1.0
50%,1296.000000,270.000000,1.0
75%,1684.750000,375.750000,1.0
max,74589.000000,17485.000000,1.0


In [9]:
!pip install pillow

In [16]:
!pip install --upgrade pillow

     ---------------------------------------- 2.5/2.5 MB 1.8 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.2.0
    Uninstalling Pillow-9.2.0:
      Successfully uninstalled Pillow-9.2.0


ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\wilbi\\Anaconda3\\Lib\\site-packages\\~il\\_imaging.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [21]:
from PIL import Image, ImageDraw, ImageFont

def My_song(song):
    img = Image.open("Logo1.png")
    font_path = "DancingScript-VariableFont_wght.ttf"
    font_size = 20
    myFont = ImageFont.truetype(font_path, font_size)
    draw = ImageDraw.Draw(img)
    text_width, text_height = draw.textsize(song, font=myFont)
    x = (500 - text_width) // 2
    y = (500 - text_height) // 2
    draw.text((x, y), song, font=myFont, fill=(255, 255, 255))
    return img

song = data.Paroles[42][:500]
image = My_song(song)
image.show()


In [22]:
#Lining up all the lyrics to create corpus
Corpus =''
for listitem in data.Paroles.astype(str):
    Corpus += listitem
    
Corpus = Corpus.lower() #converting all alphabets to lowecase 
print("Number of unique characters:", len(set(Corpus)))

Number of unique characters: 598


In [23]:
#To See all the unique characters present in the Corpus
print("The unique characters:",sorted(set(Corpus)))

The unique characters: [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\xa0', '¡', '«', '°', '´', '»', '¿', 'ß', 'à', 'á', 'ã', 'ä', 'å', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'ï', 'ñ', 'ò', 'ó', 'ö', 'ø', 'ù', 'ú', 'ü', 'ý', 'þ', 'ā', 'ă', 'ē', 'ł', 'ș', 'ț', 'ι', 'е', '\u2005', '\u200a', '\u200b', '–', '—', '‘', '’', '“', '”', '„', '•', '…', '\u202a', '\u202c', '\u205f', '\u2060', '⠀', '「', '」', 'あ', 'い', 'う', 'え', 'か', 'が', 'き', 'ぎ', 'く', 'こ', 'ご', 'さ', 'し', 'す', 'ず', 'そ', 'だ', 'ち', 'っ', 'つ', 'て', 'で', 'と', 'ど', 'な', 'に', 'の', 'は', 'ま', 'み', 'め', 'も', 'ゃ', 'ょ', 'よ', 'ら', 'り', 'る', 'れ', 'わ', 'を', 'ん', 'ィ', 'イ', 'ガ', 'シ', 'セ', 'デ', 'ナ', 'ニ', 'ハ', 'バ', 'メ', 'モ', 'ラ', 'レ', 'ロ', 'ー', '一', '世', '二', '人', '冴', '出', '君', '声', '変'

In [26]:
#Keeping only a limited set of characters. 
to_remove = ['{', '{', '|', '}', '~', '\xa0', '¡', '«', '°', '´', '»', '¿', 'ß', 'à', 'á', 'ã', 'ä', 'å', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'ï', 'ñ', 'ò', 'ó', 'ö', 'ø', 'ù', 'ú', 'ü', 'ý', 'þ', 'ā', 'ă', 'ē', 'ł', 'ș', 'ț', 'ι', 'е', '\u2005', '\u200a', '\u200b', '–', '—', '‘', '’', '“', '”', '„', '•', '…', '\u202a', '\u202c', '\u205f', '\u2060', '⠀', '「', '」', 'あ', 'い', 'う', 'え', 'か', 'が', 'き', 'ぎ', 'く', 'こ', 'ご', 'さ', 'し', 'す', 'ず', 'そ', 'だ', 'ち', 'っ', 'つ', 'て', 'で', 'と', 'ど', 'な', 'に', 'の', 'は', 'ま', 'み', 'め', 'も', 'ゃ', 'ょ', 'よ', 'ら', 'り', 'る', 'れ', 'わ', 'を', 'ん', 'ィ', 'イ', 'ガ', 'シ', 'セ', 'デ', 'ナ', 'ニ', 'ハ', 'バ', 'メ', 'モ', 'ラ', 'レ', 'ロ', 'ー', '一', '世', '二', '人', '冴', '出', '君', '声', '変', '夜', '子', '宇', '実', '巡', '広', '彩', '心', '忘', '愛', '揺', '星', '春', '月', '未', '来', '歌', '溶', '界', '的', '相', '眠', '空', '終', '色', '落', '詞', '誰', '踊', '迷', '連', '酔', '野', '雪', '静', '風', '騒', '가', '각', '간', '갈', '감', '강', '같', '개', '걍', '거', '걱', '건', '걸', '것', '게', '겐', '겠', '격', '견', '경', '계', '고', '곳', '과', '괜', '국', '군', '궁', '그', '근', '금', '기', '긴', '길', '김', '까', '깐', '께', '꼴', '꾸', '꿈', '끊', '끝', '낌', '나', '난', '날', '남', '낮', '내', '낸', '낼', '냐', '냥', '너', '넌', '널', '넘', '네', '년', '노', '놀', '놈', '높', '놔', '누', '눈', '느', '는', '늘', '늪', '니', '다', '닥', '단', '달', '닮', '답', '당', '대', '더', '던', '데', '도', '독', '돼', '되', '된', '될', '두', '둔', '둘', '둠', '뒤', '드', '든', '들', '듯', '디', '딜', '따', '딱', '때', '땐', '떠', '떡', '떨', '떼', '또', '뚫', '뜨', '뜬', '뜻', '라', '락', '란', '람', '랑', '래', '랙', '러', '런', '럼', '레', '렘', '려', '렴', '렵', '렸', '로', '론', '롭', '르', '를', '름', '리', '린', '릴', '림', '마', '만', '많', '말', '맘', '망', '맞', '맡', '매', '머', '멈', '멋', '멍', '며', '면', '모', '몫', '몰', '몸', '무', '묶', '문', '물', '뭐', '뭘', '미', '민', '믿', '바', '밖', '반', '발', '밟', '밤', '방', '백', '뱉', '버', '번', '벌', '법', '벼', '보', '복', '볼', '봐', '부', '불', '브', '블', '비', '빛', '빠', '빤', '빨', '뻔', '뻗', '사', '산', '살', '상', '새', '생', '샷', '서', '선', '설', '성', '세', '셀', '셋', '소', '속', '손', '쇠', '수', '숙', '순', '숨', '쉬', '스', '슨', '시', '식', '신', '실', '싫', '심', '싶', '싸', '싼', '써', '쓰', '쓸', '아', '악', '안', '않', '알', '애', '야', '약', '어', '얼', '없', '었', '에', '엑', '엔', '여', '영', '예', '오', '옥', '온', '올', '옵', '옷', '와', '완', '외', '우', '욱', '운', '울', '웃', '워', '원', '웬', '위', '윈', '유', '육', '으', '은', '을', '의', '이', '인', '일', '있', '잊', '자', '작', '잔', '잖', '잠', '잡', '장', '재', '저', '적', '전', '점', '정', '제', '져', '졌', '조', '좋', '주', '죽', '중', '줘', '줬', '지', '직', '진', '질', '쩔', '쩜', '쪽', '쫄', '차', '찮', '창', '찾', '채', '처', '척', '천', '청', '쳐', '추', '출', '춰', '치', '칠', '캄', '커', '컷', '콜', '크', '큼', '타', '탄', '태', '터', '통', '투', '툭', '툴', '퉁', '티', '파', '편', '폐', '포', '표', '푸', '품', '플', '픔', '피', '핑', '하', '한', '할', '함', '해', '했', '행', '헤', '황', '훈', '휘', '휩', '흐', '흔', '희', '힘', '\ufeff', '🐚', '😁', '}', '~', '©', 'à', 'á', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'ñ', 'ó', 'ö', 'ü', 'ŏ',
             'е', 'ا', 'س', 'ل', 'م', 'و', '\u2005', '\u200a', '\u200b', '–', '—', '‘', '’', '‚', '“', '”', 
             '…', '\u205f', '\ufeff', '!', '&', '(', ')', '*', '-',  '/', ]
for symbol in to_remove:
    Corpus = Corpus.replace(symbol," ")

#Corpus = re.sub("[^A-Za-z0-9'\.\n]","",Corpus) Alterneativly could be used but I want to pick and chose (:

In [27]:
#To See all the unique characters present in the Corpus
print("The unique characters:",sorted(set(Corpus)))

The unique characters: [' ', '"', '#', '$', '%', "'", '+', ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [28]:
# Storing all the unique characters present in my corpus to bult a mapping dic. 
symb = sorted(list(set(Corpus)))

L_corpus = len(Corpus) #length of corpus
L_symb = len(symb) #length of total unique characters

#Building dictionary to access the vocabulary from indices and vice versa
mapping = dict((c, i) for i, c in enumerate(symb))
reverse_mapping = dict((i, c) for i, c in enumerate(symb))

print("Total number of characters:", L_corpus)
print("Number of unique characters:", L_symb)

Total number of characters: 3601481
Number of unique characters: 52


In [29]:
#Splitting the Corpus in equal length of strings and output target
length = 40
features = []
targets = []
for i in range(0, L_corpus - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])
    
    
L_datapoints = len(targets)
print("Total number of sequences in the Corpus:", L_datapoints)

Total number of sequences in the Corpus: 3601441


**Encoding the Labels and Targets**

In [30]:
# reshape X and normalize
X = (np.reshape(features, (L_datapoints, length, 1)))/ float(L_symb)

# one hot encode the output variable
y = np_utils.to_categorical(targets)

In [ ]:
#Initialising the Model
model = Sequential()
#Adding layers
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
#Compiling the model for training  
opt = Adamax(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)

#Model's Summary               
model.summary()



In [ ]:
#Training the Model
history = model.fit(X, y, batch_size=128, epochs=100)

In [ ]:
from keras.models import load_model
#To be used later; I am saving the model 
model.save("Lyrics_Generator.h5")

In [ ]:
history_df = pd.DataFrame(history.history)
#Plotting the learnings 

fig = plt.figure(figsize=(15,4), facecolor="#B291B6")
fig.suptitle("Learning Plot of Model for Loss")
pl=sns.lineplot(data=history_df["loss"],color="#444160")
pl.set(ylabel ="Training Loss")
pl.set(xlabel ="Epochs")

**Generating the songs**

In [ ]:
# The function to generate text from model
def Lyrics_Generator(starter,Ch_count): #,temperature=1.0):
    generated= ""
    starter = starter 
    seed=[mapping[char] for char in starter]
    generated += starter 
    # Generating new text of given length
    for i in range(Ch_count):
        seed=[mapping[char] for char in starter]
        x_pred = np.reshape(seed, (1, len(seed), 1))
        x_pred = x_pred/ float(L_symb)
        prediction = model.predict(x_pred, verbose=0)[0]  
        # Getting the index of the next most probable index
        prediction = np.asarray(prediction).astype('float64')
        prediction = np.log(prediction) / 1.0 
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, prediction, 1)
        index = np.argmax(prediction)
        next_char = reverse_mapping[index]  
        # Generating new text
        generated += next_char
        starter = starter[1:] + next_char
       
    return generated

In [ ]:
#Generating a song from the model
song_1 = Lyrics_Generator("the shoe shrunk, and the school belt got ridiculously petit", 400)
#Let's have a look at the song
My_song(song_1)

In [ ]:
#Generating a song from the model using a song out of the corpus
song_2 = Lyrics_Generator("i'm a sunflower, a little funny", 400)
#Let's have a look at the song
My_song(song_2)